# TopAGNPS watershed delineation quality control

- After TopAGNPS has delineated a watershed, the final result is checked for quality control:
    - Cells coverage vs. initial THUC boundary
    - Check if delineation reached the edge of the DEM

In [14]:
import json
import geopandas as gpd

from pyagnps import topagnps, utils
from pyagnps.utils import log_to_file

## Paths to TopAGNPS runs and THUC shapefile

In [12]:
path_to_thucs = 'C:/Users/Luc/projects/pyagnps/inputs/thucs/tophuc_S_40000.gpkg'

CSA = 10 # Critical Source Area in ha
MSCL = 250 # Minimum Stream Channel Length in m
RESOLUTION = 30 # Resolution of the DEM in m
BUFFER = 500 # Buffer around the thuc in m

path_to_TOPAGNPS_bin = '/aims-nas/luc/bins/TopAGNPS_v6.00.b.026_release_64-bit_Linux' # absolute or with respect to a sub directory in path_to_dir
path_to_thucs = '/aims-nas/luc/data/tophuc_S_M_40000_closed_holes_with_container_thuc_merged_bbox_area_first_kept.gpkg'
root_dir = '/aims-nas/luc/thuc_runs_40k_SM/'
dir_runs_name = f'40000_SM_res_{RESOLUTION}_csa_{CSA}_mscl_{MSCL}_buff_{BUFFER}' # Name of the directory where the runs will be stored

thuc_dir_path = f'{root_dir}/{dir_runs_name}/'
thucid_dir_name = f'thuc_{thuc_id}_{dir_runs_name}'
thuc_topagnps_path = f'{thuc_dir_path}/{thucid_dir_name}'

path_to_qc_dir = f'{root_dir}/QualityControl/{dir_runs_name}'

thucs = gpd.read_file(path_to_thucs)

In [3]:
thuc_id = '4209'

thuc_select = thucs[thucs['tophucid']==thuc_id]
thuc_select

,tophucid,drainage_area_sqkm,outlethuc12,outlethuc12_hutype,receivinghuc12,contained_hucs,num_contained_hucs,foreign_inflows,num_foreign_inflows,tophucid_bbox_container,totophuc,bbox_area_sqkm,geometry
4203,4209,34193.84,190303031108,S,190303060803,"190303010107,190303011601,190303030104,1903030...",303,190303011505,1,None,OUTSIDE,82027.339141,"POLYGON ((-158.11651 58.75992, -158.11674 58.7..."


## Specification of the folder containing the TopAGNPS delineations and the name of the delineation run

In [4]:
path_to_cell_IDs_asc = f'{thuc_topagnps_path}/AnnAGNPS_Cell_IDs.asc'

path_to_topagnps_wrn = f'{thuc_topagnps_path}/TopAGNPS_wrn.CSV'

In [5]:
results = topagnps.quality_control_areas_vs_boundary(path_to_cell_IDs_asc, thuc_select)

In [7]:
touch_edges = topagnps.check_topagnps_wrn_log(path_to_topagnps_wrn)
results['touching_edges'] = touch_edges

In [8]:
results

{'total_cells_area_sqm': 35095540236.724976,
 'total_boundary_area_sqm': 31535840576.78877,
 'cells_missed_area_sqm': 209237441.6908982,
 'beyond_boundary_cells_area_sqm': 3768937101.626805,
 'intersection_area_sqm': 31326603135.097797,
 'fraction_boundary_covered': 0.9933650907074608,
 'fraction_boundary_missed': 0.006634909292539182,
 'touching_edges': []}

In [15]:
log_to_file(f'{path_to_qc_dir}/{thuc_id}_res_{RESOLUTION}_csa_{CSA}_mscl_{MSCL}.json', json.dumps(results, indent=2))